### Создание базы данных для библиотеки

Пусть необходимо будет хранить информацию о читателях (номер читательского билета, ФИО, адрес, телефон), хранимых книгах (шифр, автор, название, количество экземпляров), а также о выдаче книг на руки с указанием даты выдачи и срока возврата, размере штрафа в случае просрочки.

**1. Напишем скрипт для создания таблицы Читатели (readers) со следующей структурой:**  
![Скриншот](table_readers.png)

CREATE TABLE readers (  
    READER_NUM INT AUTO_INCREMENT PRIMARY KEY,  
    READER_NAME VARCHAR(100),  
    READER_ADRESS VARCHAR(1000),   
    READER_PHONE VARCHAR(100) NOT NULL  
);

**2. Напишем скрипт для создания таблицы Книги (books) со следующей структурой:**
![Скриншот](table_books.png)

CREATE TABLE books (  
    BOOK_NUM INT AUTO_INCREMENT PRIMARY KEY,  
    BOOK_AUTHOR VARCHAR(100),  
    BOOK_NAME VARCHAR(1000),  
    BOOK_COUNT INT NOT NULL DEFAULT 0  
);

**3. Напишем скрипт для создания таблицы Выдача книг(books_in_use) со следующей структурой:**  
![Скриншот](table_books_in_use.png)

В первичный ключ входят три столбца (Дата выдачи также входит в первичный ключ, потому что один и тот же читатель может брать книгу несколько раз). 

Создадим два внешних ключа при создании таблицы books_in_use:

1) Свяжет таблицу books_in_use и books по шифру книги

2) Свяжет таблицу books_in_use и readers по номеру читательского билета.

CREATE TABLE books_in_use (  
    READER_NUM INT,  
    BOOK_NUM INT,  
    ISSUE_DATE DATE,  
    RETURN_DATE DATE,  
    RETURN_PERIOD TINYINT NOT NULL DEFAULT 14,  
    FINE_AMOUNT DECIMAL(10,2) NOT NULL DEFAULT 0,  
    PRIMARY KEY(READER_NUM, BOOK_NUM, ISSUE_DATE),  
    FOREIGN KEY (BOOK_NUM) REFERENCES books(BOOK_NUM),  
    FOREIGN KEY (READER_NUM) REFERENCES readers(READER_NUM)  
);

В результате должна получиться следующая структура нашей БД:  
![Скриншот](Схема_библиотеки.png)

**4. Заполним таблицу readers следующими данными:**  
![Скриншот](table_readers_2.png)

INSERT INTO readers (reader_name, reader_adress, reader_phone)  
VALUES   
('Сидоров', 'ул. Ленина, 5а', '4424556'),  
('Ванюшкин', 'ул. Космонавтов, д. 31, кв. 143', '4545222'),  
('Дроздов', 'ул. Ленина, д. 3, кв. 13', '8955454'),  
('Голубева', 'ул. Тимирязева, д. 35, кв. 18', '5454555'),  
('Шишкин', 'ул. Революции, д. 16/7, кв. 45', '454564564'),  
('Книголюбова', 'ул. Пушкина, д. 38', '54664545'),  
('Петров', 'ул. Пушкина, д. 31, кв. 16', '6115646'),  
('Паринова', null, '46488484'),  
('Птичкина', 'ул. Зеленая, д. 3/7', '65545445'),  
('Дроздов', 'ул. Конструкторов, д. 89, кв. 14', '546544');

**5. Заполним таблицу books следующими данными:**  
![Скриншот](table_books_2.png)

INSERT INTO books (book_author, book_name, book_count)  
VALUES  
('Толстой', 'Война и мир', 15),  
('Достоевский', 'Идиот', 13),  
('Пушкин', 'Евгений Онегин', 18),  
('Пушкин', 'Руслан и Людмила', 5),  
('Пушкин', 'Медный всадник', 11),  
('Барто', 'Стихи детям', 1),  
('Чехов', 'Вишневый сад', 8),  
('Чехов', 'Дядя Ваня', 7),  
('Тургенев', 'Отцы и дети', 13),  
('Тургенев', 'Муму', 4);

**6. Заполним таблицу books_in_use следующими данными:**  
Для заполнения books_in_use будем использовать DEFAULT-значения для столбцов return_period и fine_amount, а не явное заполнение числами.  
![Скриншот](table_books_in_use_2.png)

INSERT INTO books_in_use (reader_num, book_num, issue_date, return_date)  
VALUES  
(1, 1, '2023-09-15', '2023-10-17'),  
(1, 8, '2023-10-17', null),  
(2, 1, '2023-10-04', '2023-10-16'),  
(3, 2, '2023-09-11', '2023-09-30'),  
(3, 4, '2023-09-11', '2023-09-30'),  
(3, 5, '2023-09-11', '2023-09-30'),  
(4, 1, '2023-09-28', '2023-10-05'),  
(4, 3, '2023-09-28', '2023-10-05'),  
(4, 8, '2023-10-05', '2023-10-31'),  
(5, 6, '2023-09-14', '2023-10-14'),  
(6, 1, '2023-09-09', '2023-09-20'),  
(6, 1, '2023-09-20', '2023-10-01'),  
(7, 1, '2023-09-13', '2023-09-21'),  
(7, 7, '2023-09-21', '2023-10-20'),  
(8, 7, '2023-09-11', null);

**7. Заполним столбец fine_amount (сумма штрафа) в таблице books_in_use следующим образом:** 

За каждый день просрочки (дни, которые превышают return_period) начисляется штраф в размере 8,45 р.    

Количество дней просрочки определяется как разница между Датой возврата книги и Датой выдачи   

Если дата возврата не заполнена, значит книга еще в использовании, штраф не начисляем.

SET SQL_SAFE_UPDATES = 0;  
UPDATE books_in_use  
SET fine_amount = (DATEDIFF(return_date, issue_date) - return_period) * 8.45  
WHERE return_date IS NOT NULL  
  AND DATEDIFF(return_date, issue_date) > return_period;  
SET SQL_SAFE_UPDATES = 1;

Получившееся таблица readers имеет следующий вид:  
![Скриншот](table_readers_3.png)

Получившееся таблица books имеет следующий вид:
![Скриншот](table_books_3.png)

После обновления столбца Сумма штрафа таблица books_in_use имеет следующий вид:  
![Скриншот](table_books_in_use_3.png)